## Vibe Matcher 

# Why AI at Nexora  
At Nexora, Artificial Intelligence can leveraged to bridge human emotion with data-driven personalization. Fashion, being inherently expressive, demands technology that can understand vibes not just words. This project, “Vibe Matcher”, demonstrates how AI can capture a user’s aesthetic intent and translate it into relevant product recommendations. By combining text embeddings, semantic similarity, and real-time vector search, Nexora can showcase how machine learning transforms traditional product catalogs into intelligent, experience driven systems that understand how users feel, not just what they type.




## 0) Install & imports

In [19]:
import json, numpy as np, pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


## 1) Data Prep

In [20]:

CATALOG_PATH = "vibe_products.csv"

df = pd.read_csv(CATALOG_PATH)
assert {"name","desc","vibes"}.issubset(df.columns), "CSV must contain 'name', 'desc', 'vibes' columns."
df.head(3)


,id,name,desc,vibes
0,1,Boho Breeze Maxi Dress,Flowy maxi dress in earthy neutrals with tasse...,"['boho', 'free-spirited', 'festival', 'cozy']"
1,2,City Sprint Windbreaker,Lightweight windbreaker with reflective trims ...,"['athleisure', 'streetwear', 'urban', 'energet..."
2,3,Monochrome Power Blazer,"Sharp, structured blazer in deep black with mi...","['minimal', 'formal', 'monochrome', 'chic']"


## 2) Embeddings

In [ ]:
# since openAPI is now not available for free trial also I have used 
# Transformer-based model thenlper/gte-large 

MODEL_NAME = "thenlper/gte-large"
model = SentenceTransformer(MODEL_NAME)

def embed_texts(texts):
    return model.encode(texts, show_progress_bar=True)

descs = df["desc"].astype(str).tolist()
product_vecs = embed_texts(descs)
df["vector"] = [v.tolist() for v in product_vecs]

# Save for re-use
df.to_parquet("vibe_catalog_with_embeddings.parquet", index=False)
print("✅ Created product embeddings. Dim =", len(df["vector"].iloc[0]))
df.head(2)


Batches: 100%|██████████| 1/1 [00:00<00:00,  3.21it/s]

✅ Created product embeddings. Dim = 1024


,id,name,desc,vibes,vector
0,1,Boho Breeze Maxi Dress,Flowy maxi dress in earthy neutrals with tasse...,"['boho', 'free-spirited', 'festival', 'cozy']","[-0.030112499371170998, 0.0014987626345828176,..."
1,2,City Sprint Windbreaker,Lightweight windbreaker with reflective trims ...,"['athleisure', 'streetwear', 'urban', 'energet...","[-0.023059139028191566, -0.007893639616668224,..."


## 3) Vector Search Sim (Cosine Sim)

In [22]:

TOP_K = 3
LOW_CONF_THRESH = 0.50

def stack_vectors(series):
    arrs = [np.array(v, dtype=np.float32) for v in series]
    return np.vstack(arrs)

def search(query_text, df, top_k=TOP_K):
    query_text = (query_text or "").strip()
    if not query_text:
        return None, None, "Empty query. Please describe a vibe (e.g., 'energetic urban chic')."
    q_vec = model.encode([query_text])[0].reshape(1, -1)
    mat = stack_vectors(df["vector"])
    sims = cosine_similarity(q_vec, mat)[0]
    idx = np.argsort(-sims)[:top_k]
    results = df.iloc[idx][["id","name","desc","vibes"]].copy().reset_index(drop=True)
    results["similarity"] = [float(sims[i]) for i in idx]
    best = float(np.max(sims))
    warn = None
    if best < LOW_CONF_THRESH:
        warn = f"No strong match (best score {best:.2f} < {LOW_CONF_THRESH}). Try refining your vibe (e.g., 'sporty night streetwear', 'cozy winter chic')."
    return results, best, warn


## 4) Try a query

In [26]:

query = "Retro Grace"  
results, best, warn = search(query, df, top_k=3)
print("Query:", query)
if warn: print("⚠️", warn)
results


Query: Retro Grace


,id,name,desc,vibes,similarity
0,8,Retro Court Sneakers,Low-top leather sneakers with clean lines and ...,"['casual', 'streetwear', 'retro', 'urban']",0.822742
1,7,Silk Slip Moonlight,Bias-cut silk slip in charcoal with a subtle s...,"['chic', 'minimal', 'evening', 'elegant']",0.791723
2,5,Denim Revival Jacket,Boxy mid-wash denim jacket with distressed sea...,"['streetwear', 'casual', 'urban', 'grunge-lite']",0.785035


## 5) Test & Eval

In vibe_matcher_evaluation.ipynb
